# problem


create data set randomly

In [423]:
import numpy as np
import pandas as pd

np.random.seed(42)

num_customers = 20

customer_coords = np.random.rand(num_customers, 2) * 100

depot_coords = np.array([[50, 50]])

all_coords = np.vstack([depot_coords, customer_coords])

coords_df = pd.DataFrame(all_coords, columns=['x', 'y'])

coords_df.to_csv('vrp_data.csv', index=False)

print(coords_df)

            x          y
0   50.000000  50.000000
1   37.454012  95.071431
2   73.199394  59.865848
3   15.601864  15.599452
4    5.808361  86.617615
5   60.111501  70.807258
6    2.058449  96.990985
7   83.244264  21.233911
8   18.182497  18.340451
9   30.424224  52.475643
10  43.194502  29.122914
11  61.185289  13.949386
12  29.214465  36.636184
13  45.606998  78.517596
14  19.967378  51.423444
15  59.241457   4.645041
16  60.754485  17.052412
17   6.505159  94.888554
18  96.563203  80.839735
19  30.461377   9.767211
20  68.423303  44.015249


# encoding

In [424]:
import numpy as np
import pandas as pd

coords_df = pd.read_csv('vrp_data.csv')

all_coords = coords_df.values

customer_indices = np.arange(0, len(all_coords))

customer_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [425]:
import plotly.express as px

fig = px.scatter(coords_df, x='x', y='y', color=['Depot'] + ['Customer']*num_customers, 
                 labels={'color': 'Location Type'}, title='Customer and Depot Locations')

fig.show()


# huristic function

distance calculated using eculadian distance formula with L2 norm

In [426]:
def distance(route, df):
    total_distance = 0
    coords = df[['x', 'y']].values

    depot = coords[0]

    total_distance += np.linalg.norm(depot - coords[route[0]])

    for i in range(len(route) - 1):
        total_distance += np.linalg.norm(coords[route[i]] - coords[route[i + 1]])

    total_distance += np.linalg.norm(coords[route[-1]] - depot) # to calculate distance from last customer to depot

    return total_distance


# customer_indices = np.arange(0, len(coords_df))

# example_route = np.random.permutation(customer_indices[1:])  
# example_route = np.insert(example_route, 0, 0)  
# total_cost = distance(example_route, coords_df)
# print("Example Route: ", example_route)
# print("Total Cost of the Example Route: ", total_cost)

In [427]:
num_customers = len(coords_df) - 1

population_size = 100

customer_indices = np.arange(1, num_customers + 1) 

population = []
for _ in range(population_size):
    route = np.random.permutation(customer_indices)  
    route = np.insert(route, 0, 0)  
    population.append(route)

population = np.array(population)

for i in range(5):
    print(f"Route {i + 1}: {population[i]}")

pd.DataFrame(population).to_csv('initial_population.csv', index=False)

Route 1: [ 0 16 17  5 10  9 20 11  6 19 13 18  2  4  1 14  3 15  8 12  7]
Route 2: [ 0  3  1  6 15 19 20 11 17  5 13  9  7  8 16  4 14 10  2 12 18]
Route 3: [ 0  2  5  6  4 11 17  3 14 10 12 16  8 18  7 19  1 20 15  9 13]
Route 4: [ 0  4 14 16  3 19  6 13 11 20  2  8 15 17 12  9 18  7 10  5  1]
Route 5: [ 0 12  8  9  4  6  2 13 16 11  7 10 17 19 18 14 15  5 20  1  3]


In [428]:
population

array([[ 0, 16, 17, ...,  8, 12,  7],
       [ 0,  3,  1, ...,  2, 12, 18],
       [ 0,  2,  5, ..., 15,  9, 13],
       ...,
       [ 0, 11, 13, ...,  4, 16, 20],
       [ 0, 16, 12, ...,  4, 14, 19],
       [ 0, 18, 13, ..., 20, 10,  3]])

# selection function

In [429]:
import random

def selection(population, fitness_scores, num_parents):
    sorted_indices = np.argsort(fitness_scores)
    ranked_population = [population[i] for i in sorted_indices]
    selected_parents = ranked_population[:num_parents]
    return selected_parents

<h1>evlolve </h1>
<p>Something </p>

<style>
    h1 {
        color: red;
    }
    p {
        position: relative;
        animation: moveRightLeft 2s infinite ease-in-out;
    }

    @keyframes moveRightLeft {
        0% {
            left: 0;
        }
        50% {
            left: 100px;
        }
        100% {
            left: 0;
        }
    }
</style>

In [430]:
def position_based_crossover(parent1, parent2):
    size = len(parent1)
    child1, child2 = np.zeros(size, dtype=int), np.zeros(size, dtype=int)
    child1[0], child2[0] = 0, 0  # Ensure depot at start and end

    # Select random positions for crossover
    positions = np.random.choice(range(1, size-1), size//3, replace=False)

    # Copy positions from parents to children
    child1[positions] = parent1[positions]
    child2[positions] = parent2[positions]

    # Fill remaining positions in children
    fill_positions(child1, parent2, positions)
    fill_positions(child2, parent1, positions)

    return child1, child2

def fill_positions(child, parent, positions):
    size = len(child)
    fill_pos = 1
    for i in range(1, size-1):
        if fill_pos >= size-1:
            break
        if parent[i] not in child:
            while fill_pos in positions or child[fill_pos] != 0:
                fill_pos += 1
            child[fill_pos] = parent[i]

In [431]:
def evolve(population, df, retain=0.2, random_select=0.05, mutate=0.01):
    fitness_scores = [distance(ind, df) for ind in population]
    
    retain_length = int(len(population) * retain)
    parents = selection(population, fitness_scores, retain_length)
    
    for individual in population[retain_length:]:
        if random_select > np.random.rand():
            parents.append(individual)
    
    # swap two random customers in each individual for mutation function 
    for individual in parents:
        if mutate > np.random.rand():
            pos1 = np.random.randint(1, len(individual))
            pos2 = np.random.randint(1, len(individual))
            individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
    
    parents_length = len(parents)
    desired_length = len(population) - parents_length

    children = []
    while len(children) < desired_length:
        male_idx = np.random.randint(0, parents_length)
        female_idx = np.random.randint(0, parents_length)
        if male_idx != female_idx:
            male = parents[male_idx]
            female = parents[female_idx]
            child1, child2 = position_based_crossover(male, female)
            children.append(child1)
            if len(children) < desired_length:
                children.append(child2)

    parents.extend(children)
    return parents

# trying the alogrithm 

In [432]:
# generations = 100
# for _ in range(generations):
#     population = evolve(population, coords_df)


# fitness_scores = [distance(ind, coords_df) for ind in population]
# best_route_idx = np.argmin(fitness_scores)
# best_route = population[best_route_idx]
# print("Best Route: ", best_route)
# print("Best Route Distance: ", fitness_scores[best_route_idx])

In [433]:
import plotly.express as px
import plotly.graph_objects as go

# Run the genetic algorithm
generations = 30
for _ in range(generations):
    population = evolve(population, coords_df)

# Find the best route
fitness_scores = [distance(ind, coords_df) for ind in population]
best_route_idx = np.argmin(fitness_scores)
best_route = population[best_route_idx]
print("Best Route: ", best_route)
print("Best Route Distance: ", fitness_scores[best_route_idx])

fig = px.scatter(coords_df, x='x', y='y', color=['Depot'] + ['Customer']*num_customers, 
                 labels={'color': 'Location Type'}, title='Customer and Depot Locations')

route_coords = coords_df.iloc[best_route]

fig.add_trace(
    go.Scatter(
        x=route_coords['x'], 
        y=route_coords['y'], 
        mode='lines+markers', 
        name='Best Route'
    )
)

fig.show()

Best Route:  [ 0  5 13  9 14 12  8  3 19 10 16 11 15  7 20  2 18  1 17  4  0]
Best Route Distance:  457.60873426927367


# solution with plot 

In [434]:
import plotly.graph_objects as go

generations = 100
best_distances = []

for _ in range(generations):
    population = evolve(population, coords_df)
    fitness_scores = [distance(ind, coords_df) for ind in population]
    best_distances.append(np.min(fitness_scores))

# Plotting the loss curve

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(generations)), y=best_distances, mode='lines', name='Best Distance'))

fig.update_layout(title='Best Route Distance Over Generations',
                  xaxis_title='Generation',
                  yaxis_title='Distance')

fig.show()

fitness_scores = [distance(ind, coords_df) for ind in population]
best_route_idx = np.argmin(fitness_scores)
best_route = population[best_route_idx]
print("Best Route: ", best_route)
print("Best Route Distance: ", fitness_scores[best_route_idx])

Best Route:  [ 0  5 13  9 14 12  8  3 19 10 16 11 15  7 20  2 18  1 17  4  0]
Best Route Distance:  457.60873426927367
